# Setting composition constraints in CompositionSpace

This notebook demonstrates two legal methods to set composition constraints in CompositionSpace.

In [1]:
from pymatgen.core import Species
from smol.cofe.space.domain import Vacancy
from smol.moca.composition.space import CompositionSpace

In [2]:
# A spinel chemical space.
li = Species("Li", 1)
mn2 = Species("Mn", 2)
mn3 = Species("Mn", 3)
mn4 = Species("Mn", 4)
va = Vacancy()
o = Species("O", -2)
f = Species("F", -1)
bits = [[li, mn2, mn3, mn4, va], [li, mn2, va], [o, f]]
sl_sizes = [1, 2, 1]

### 1) String format
The string format is a more programmatic and more user-friendly way to specify composition constraints. Legal constraint strings to be set in other_constraints arguments are shown below:

In [3]:
# The charge-balance constraint is included by default, as well as site number conservation.
space = CompositionSpace(bits, sl_sizes, 
                         other_constraints = ["Li+ + Mn2+ == 3 Mn4+",
                                              "2 Mn2+(1) <= 1 Li+(0) +2 F-(2)",
                                              "O2- +2 F- >= 1"])
print("Equality constraints:")
print("A:", space._A)
print("b:", space._b)
print("Inequality constraints (<=):")
print("A_leq:", space._A_leq)
print("b_leq:", space._b_leq)

Equality constraints:
A: [[ 1  2  3  4  0  1  2  0 -2 -1]
 [ 1  1  1  1  1  0  0  0  0  0]
 [ 0  0  0  0  0  1  1  1  0  0]
 [ 0  0  0  0  0  0  0  0  1  1]
 [ 1  1  0 -3  0  1  1  0  0  0]]
b: [0 1 2 1 0]
Inequality constraints (<=):
A_leq: [[-1  0  0  0  0  0  2  0  0 -2]
 [ 0  0  0  0  0  0  0  0 -1 -2]]
b_leq: [ 0 -1]


In the constraint strings above:

"Li+ + Mn2+ == 3 Mn4+" means the total amount of species Li+ on all sublattices and the total amount of species Mn2+ on all sublattices should sum up to be equal to 3 times of the total amount of species Mn4+ on all sublattices.

"2 Mn2+(1) <= 1 Li+(0) +2 F-(2)" means 2 times the total amount of Mn2+ on sublattice No. 1 should be less or equal to the total amount of Li+ on sublattice No. 0 plus 2 times the total amount of F- on sublattice No. 2. The integer number in the bracket closely following the species string specifies the index of sub-lattice to be constrained.

"O2- + 2 F- >= 1" means IN A PRIMIVIE CELL, the total amount of O2- on all sublattices plus 2 times the amount of F- on all sublattices should sum up to be larger or equals to 1.

Also note that:

    1, No space is allowed within a species string.
    
    2, No space is be allowed between the species string and the bracket enclosed index.
    
    3, The relationship operator ("=", "==", "<=" or ">=") must have one space in the front and one space in the back.
    
    4, Every species string (including the bracket closed sub-lattice index, if any) must have one space in the front and one space in the back unless it is located at the beginning or the end of the constraint string.
    
    5, The number 1 in +1 or -1 can be omitted.

### 2) Vector format
Constraints can also be represented in a vector format, where each constraint is given by a tuple of a vector, a number and a string. The vector represents the pre-factor of the amount of each species on each sub-lattice to the left-hand side of the constraint equation, ordered as simply concatenating sublists in CompositionSpace.bits. (In this example, ordered as: Li+(0), Mn2+(0), Mn3+(0), Mn4+(0), Vacancy(0), Li+(1), Mn2+(1), Vacancy(1), O2-(2), F-(2).)

The number represents the right-handside of the constraint equation. The string represents the relationship between the left and the right-hand side ("leq","eq" or "geq"). 

Note that all constraints must be written assuming the system size is confined to a single primitive cell. Geq(>=) constraints are always transformed into Leq(<=) and stored as Leq in the CompositionSpace object.

For example, the constraints in section 1) can be equivalently given as:

In [4]:
other_constraints = [
    ([1, 1, 0, -3, 0, 1, 1, 0, 0, 0], 0, "eq"),
    ([-1, 0, 0, 0, 0, 0, 2, 0, 0, -2], 0, "leq"),
    ([0, 0, 0, 0, 0, 0, 0, 0, 1, 2], 1, "geq"),
]
# The charge-balance constraint is included by default, as well as site number conservation.
space = CompositionSpace(bits, sl_sizes, 
                         other_constraints = other_constraints)
print("Equality constraints:")
print("A:", space._A)
print("b:", space._b)
print("Inequality constraints (<=):")
print("A_leq:", space._A_leq)
print("b_leq:", space._b_leq)

Equality constraints:
A: [[ 1  2  3  4  0  1  2  0 -2 -1]
 [ 1  1  1  1  1  0  0  0  0  0]
 [ 0  0  0  0  0  1  1  1  0  0]
 [ 0  0  0  0  0  0  0  0  1  1]
 [ 1  1  0 -3  0  1  1  0  0  0]]
b: [0 1 2 1 0]
Inequality constraints (<=):
A_leq: [[-1  0  0  0  0  0  2  0  0 -2]
 [ 0  0  0  0  0  0  0  0 -1 -2]]
b_leq: [ 0 -1]


One is also allowed to mix these two formats in the list passed into other_constraints.